In [1]:
# Import necessary Libraries
import mxnet as mx
from mxnet import nd, autograd, gluon
import numpy as np
import h5py
#import logging
#logging.getLogger().setLevel(logging.INFO)

/home/ec2-user/anaconda3/envs/mxnet_p36/lib/python3.6/site-packages/urllib3/contrib/pyopenssl.py:46: DeprecationWarning: OpenSSL.rand is deprecated - you should use os.urandom instead
  import OpenSSL.SSL


In [2]:
# Import the Dataset
dataset = h5py.File('datasets/datasets.h5', 'r')
train_set_x_orig = np.array(dataset['train_set_x'][:])
train_set_y_orig = np.array(dataset['train_set_y'][:])
test_set_x_orig = np.array(dataset['test_set_x'][:])
test_set_y_orig = np.array(dataset['test_set_y'][:])

In [9]:
# View Dataset
print("Training Data Dimensions: {}".format(train_set_x_orig.shape))
print("No. Examples: {}".format(train_set_x_orig.reshape((-1, 12288)).shape[0]))
print("No. Inputs: {}".format(train_set_x_orig.reshape((-1, 12288)).shape[1]))

Training Data Dimensions: (209, 64, 64, 3)
No. Examples: 209
No. Inputs: 12288


In [54]:
# Standardize Images
def transform(x):
    x = x.reshape((x.shape[0], -1))
    return x.astype(np.float32)/255
train_x = transform(train_set_y_orig)
train_y = train_set_y_orig.astype(np.float32)
test_x = transform(test_set_x_orig)
test_y = test_set_y_orig.astype(np.float32)

In [65]:
# Configure the Dataset
batch_size = 209 # Using full set of observations
num_inputs = train_set_x_orig.reshape((-1, 12288)).shape[1]
num_examples = train_set_x_orig.reshape((-1, 12288)).shape[0]
num_outputs = 1
net_dims = [20, 7, 5, 1]

# Create Training and Test Data
train_data = mx.gluon.data.DataLoader(
    mx.gluon.data.ArrayDataset(
        train_x,
        train_y
    ),
    shuffle=False,
    batch_size=batch_size
)
test_data = mx.gluon.data.DataLoader(
    mx.gluon.data.ArrayDataset(
        test_x,
        test_y
    ),
    shuffle=False,
    batch_size=50
)

In [66]:
# Network Model
net = gluon.nn.Sequential()
with net.name_scope():
    net.add(gluon.nn.Dense(net_dims[0], activation='relu'))
    net.add(gluon.nn.Dense(net_dims[1], activation='relu'))
    net.add(gluon.nn.Dense(net_dims[2], activation='relu'))
    net.add(gluon.nn.Dense(net_dims[3]))

In [ ]:
# Parameter Initialization
net.collect_params